## wiki closure date
(under construction)

In [ ]:
closed_content_wikis = public_content_dbs.query("""status == 'closed'""")

In [ ]:
closed_content_wikis.git_closed_dt.value_counts()

In [ ]:
db_group_prefix_map = {'wikipedia': 'Wp',   
                       'wikibooks': 'Wb',
                       'wiktionary': 'Wt', 
                       'wikiquote': 'Wq', 
                       'wikisource': 'Ws',
                       'wikinews': 'Wn',
                       'wikivoyage': 'Wy',
                       'wikiversity': 'Wv'}

In [ ]:
def generate_prefix(db_group, language_code, prefix_map=db_group_prefix_map):
    return f'{db_group_prefix_map[db_group]}/{language_code}'

In [ ]:
closed_content_wikis['prefix'] = closed_content_wikis[['database_group', 'language_code']].apply(lambda x:generate_prefix(x.database_group, x.language_code), axis=1)

In [ ]:
import_log_query = """
WITH 
    logs AS (
        SELECT 
            log_id,
            log_timestamp,
            log_title,
            REGEXP_SUBSTR(log_title, 'W[a-z]/[a-z]+') AS prefix
        FROM 
            logging
        WHERE 
            log_type = 'import'
        HAVING
            REGEXP_SUBSTR(log_title, 'W[a-z]/[a-z]+') IN {CLOSED_DBS}),
    
    first_log AS (
        SELECT
            MIN(log_timestamp) AS log_timestamp,
            prefix
        FROM
            logs
        GROUP BY
            prefix)
    
SELECT
    prefix,
    CONCAT(YEAR(log_timestamp), '-', MONTH(log_timestamp), '-', DAY(log_timestamp)) AS first_log_timestamp
FROM 
    first_log
"""

incubator_import_log = wmf.mariadb.run(import_log_query.format(CLOSED_DBS=sql_tuple(closed_content_wikis.prefix.values)), dbs='incubatorwiki')
incubator_import_log.head()

In [ ]:
incubator_import_log.shape

In [ ]:
closed_content_wikis = pd.merge(closed_content_wikis, incubator_import_log, on='prefix', how='left')
closed_content_wikis.head()

In [ ]:
close_missing = closed_content_wikis[closed_content_wikis.first_log_timestamp.isna()]
close_missing

In [ ]:
response = requests.get('https://meta.wikimedia.org/wiki/Stewards/Former_stewards')
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
former_stewards = pd.read_html(str(soup.find('table', {'class': 'sortable'})))[0]

In [ ]:
closed_content_wikis.shape

In [ ]:
current_stewards = wmf.mariadb.run("""
SELECT *
FROM global_user_groups ug
JOIN globaluser u
ON u.gu_id = ug.gug_user
WHERE gug_group = 'steward'


""", ['centralauth'], use_x1=True)

In [ ]:
stewards = former_stewards.Username.values.tolist() + current_stewards.gu_name.values.tolist()

In [ ]:
stewards_sql = sql_tuple(stewards)

In [ ]:
df = wmf.mariadb.run(f"""
WITH max_time AS (SELECT MAX(rev_timestamp) AS timestamp
FROM revision r
JOIN actor a
ON r.rev_actor = a.actor_id
WHERE NOT (actor_name IN {stewards_sql} OR actor_name IN ('Flow talk page manager', 'Global rename script', 'MediaWiki message delivery', 'Maintenance script')))

SELECT * FROM revision r
JOIN actor a ON r.rev_actor = a.actor_id
JOIN max_time m WHERE r.rev_timestamp = m.timestamp


""", ['abwiktionary'])


df




In [ ]:
warnings.filterwarnings('ignore')
df = wmf.mariadb.run(f"""
SELECT *, MAX(rev_timestamp) AS timestamp
FROM revision r
JOIN actor a
ON r.rev_actor = a.actor_id
WHERE actor_name NOT IN {stewards_sql}


""", close_missing.database_code.values.tolist())

df['wiki_db'] = close_missing.database_code.values.tolist()
df




In [ ]:
df.actor_name.unique()